In [ ]:
!pip install slack_sdk
!pip install qdrant_client
!pip install duckduckgo-search

In [ ]:

from dotenv import find_dotenv, load_dotenv

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

qdrant_client = QdrantClient(host="localhost", port=6333)
qdrant = Qdrant(
    client=qdrant_client,
    collection_name="SlackMessages",
    content_payload_key="text",
    embeddings=embeddings,
)
retriever = qdrant.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-4", temperature=0)
retrieval_qa = RetrievalQA.from_chain_type(llm, retriever=retriever)

In [ ]:
from langchain.agents import initialize_agent, Tool, tool
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()


tools = [
    Tool(
        name="Search",
        func=retriever.get_relevant_documents,
        description="use this tool initially to gain context used for follow-up lookups",
    ),
    Tool(
        name="Lookup",
        func=search,
        description="the only tool for searching the internet for facts or uncovering missing information",
    ),
]

react = initialize_agent(
    tools,
    llm,
    early_stopping_method="generate",
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
react.run("Should we upgrade TypeScript in our repo? If not, why?")